In [4]:
import numpy as np
import pandas as pd
from astropy.table import Table, vstack
from matplotlib import pyplot as plt
from glob import glob
from astropy.stats import LombScargle
from scipy import stats
from scipy.optimize import curve_fit
import warnings
from TESStools import *
import celerite
from celerite import terms
from scipy.optimize import minimize

In [2]:
paper_tics = [179305185,404850274,279957111,389437365,389565293,179206253,40404470,29984014]

In [13]:
def noise_func(f, alpha_0, tau, gamma, alpha_w):
    return alpha_0 / (1.0 + np.power((2.0*np.pi*tau*f),gamma)) + alpha_w

In [14]:
def log_noise(f, alpha_0, tau, gamma, alpha_w):
    return np.log10(noise_func(f, alpha_0, tau, gamma, alpha_w))

In [43]:
header = ''

for i, tid in enumerate(paper_tics):
    
    #name and light curve
    fname = np.unique(massive['CommonName'][massive['ticid']==tid])[0].replace('*','').replace('V','')
    print(fname)
    lc, lc_smooth = lc_extract(get_lc_from_id(tid), smooth=128)
    
    #normalization
    if (fname == 'HD 268687'):
        lc, p = polynorm(lc, deg=7)
        time, flux, err = lc['Time'], lc['NormFlux'], lc['NormErr']
        print(time[0],flux[0],err[0])
    else:
        time, flux, err = lc['Time'], lc['Flux'], lc['Err']
    
    pseudo_NF = 0.5 / (np.mean(np.diff(time)))
    rayleigh = 1.0 / (np.max(time)-np.min(time))
    
    #calculate the ls object for white noise and red noise calculations (red noise is in power space)
    ls_white = LombScargle(time, flux)
    ls_red = LombScargle(time, flux, normalization='psd')
    f_w, p_w = ls_white.autopower(minimum_frequency=1.0/30.0,
                    maximum_frequency=pseudo_NF)
    f_r, p_r = ls_red.autopower(minimum_frequency=1.0/30.0,
                    maximum_frequency=pseudo_NF)
    
    #find power where FAL < 0.01
    probabilities = [0.01]
    fal = ls_white.false_alarm_level(probabilities)[0]
    
    #fit the psd periodogram with our noise function
    p0 = [np.max(p_r), 1.0, 1.0, p_r[np.argmax(f_r)]]
    bounds = (-np.inf,np.inf)
    popt, pcov = curve_fit(log_noise, f_r, np.log10(p_r), bounds=bounds, p0=p0)
    
    #now, the integral of the noise function should be the variance of the flux * f_Ny
    test_fs = np.logspace(np.log10(1.0/30.0),np.log10(pseudo_NF),1e5)
    red_int = np.trapz(log_noise(test_fs,*popt),test_fs)
    red_scale = np.var(flux)*pseudo_NF/red_int
    
    good_fs, good_amps, good_phases = prewhiten(time, flux, err)
    
    noise = np.std(flux)
    snr = good_amps[:,0]/noise
    
    white_sig = np.array([p_w[np.argmin(np.abs(freq-f_w))] > fal for freq in good_fs[:,0]])
    
    red_Z = np.array([amp**2.0*len(time) / ((2.0*red_scale*log_noise(f,*popt))**2.0) for amp,f in zip(good_amps[:,0],good_fs[:,0])])
    red_probs = 1.0 - np.exp(-np.exp(-0.93*red_Z+np.log(0.8*len(time))))
    red_sig = red_probs < 0.01
    print('All freq RN significant? {}'.format(np.all(red_sig)))
    
    header += r"""\begin{{deluxetable*}}{{cccccccc}}
\tabletypesize{{\footnotesize}}
\tablecaption{{Unique frequencies, amplitudes, phases, and formal errors for {0} found via prewhitening. For each frequency, we specify the signal to noise as defined in text, and whether the corresponding signal is significant under the assumption of white noise.\label{{tab:{0}_freqs}}}}
\tablehead{{\colhead{{$f_j$}} & \colhead{{$\epsilon(f_j)$}} & \colhead{{$A_j$}} & \colhead{{$\epsilon(A_j)$}} & \colhead{{$\phi_j$}} &  \colhead{{$\epsilon(\phi_j)$}} & \colhead{{SNR}} & \colhead{{White Noise Significant?}}\\
\colhead{{[day$^{{-1}}$]}} & \colhead{{[day$^{{-1}}$]}} & \colhead{{[ppt]}} & \colhead{{[ppt]}} & \colhead{{[radians]}} & \colhead{{[radians]}} & \colhead{{}} & \colhead{{}} }} 
\startdata
""".format(fname)
    for f,a,p,s,w,r in zip(good_fs,good_amps,good_phases,snr,white_sig,red_sig):
        
        if w:
            wn = 'Y'
        else:
            wn = 'N'
        if r:
            rn = 'Y'
        else:
            rn = 'N'
            
        header += "${0:.8f}$ & ${1:.8f}$ & ${2:.8f}$ & ${3:.8f}$ & ${4:.4f}$ & ${5:.4f}$ & {6:.4f} & {7} \\\\ \n".format(f[0],f[1],a[0]*1e3,a[1]*1e3,p[0],p[1],s,wn).replace('e','\\times10^{')
    header.rstrip()
    header.rstrip('\\')
    header += r"""
\enddata
\end{deluxetable*}

"""
    
with open("appendix_deluxetable.txt", "w") as text_file:
        text_file.write(header)

 S Dor


/Users/carina/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: RuntimeWarning: invalid value encountered in log10
  from ipykernel import kernelapp as app


f_Ny = 322.8060468379108, f_R = 0.0177879072510214
10
20
30
40
50
60
70
Found 71 frequencies
41 unique frequencies
All freq RN significant? True
HD 269953


/Users/carina/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: RuntimeWarning: invalid value encountered in log10
  from ipykernel import kernelapp as app
/Users/carina/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: RuntimeWarning: invalid value encountered in power
  from ipykernel import kernelapp as app


f_Ny = 323.89171072083246, f_R = 0.01778794028727421
10
Found 14 frequencies
14 unique frequencies
All freq RN significant? True
HD 269582


...

f_Ny = 323.7577400015701, f_R = 0.017787909455610686
10
Found 11 frequencies
7 unique frequencies
All freq RN significant? True
HD 268687


/Volumes/shoobert/Research/UW/TESS/code/TESStools.py:111: RankWarning: Polyfit may be poorly conditioned
  p = np.polyfit(lc['Time'].values,lc['Flux'].values,deg=deg)


1325.2963953179183 1.0037015815853616 0.0012825290463256078


/Users/carina/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: RuntimeWarning: invalid value encountered in log10
  from ipykernel import kernelapp as app


f_Ny = 323.83746567083887, f_R = 0.017787891882718898
10
20
30
40
50
60
Found 64 frequencies
33 unique frequencies
All freq RN significant? True
